In [164]:
from aoc import *
from copy import deepcopy
from collections import defaultdict, Counter, deque
import re
from z3 import Ints, Solver, sat
from tqdm import tqdm

year = 2024
day = 20

download_input(year, day)

In [165]:
aoc, lines, G, R, C = read_input(day, test=False)

for r, row in enumerate(G):
    for c, cell in enumerate(row):
        if cell == "S":
            s = (r, c)
            G[r][c] = "."
        if cell == "E":
            e = (r, c)
            G[r][c] = "."

print(s, e)

['#############################################################################################################################################', '#.......###...#...###.....###...#...#.......#...#.............###.......#...#####.........#.....#...........###.......#...#.....#...###...###', '#.#####.###.#.#.#.###.###.###.#.#.#.#.#####.#.#.#.###########.###.#####.#.#.#####.#######.#.###.#.#########.###.#####.#.#.#.###.#.#.###.#.###', '#.#...#.....#...#...#...#.....#.#.#.#...#...#.#.#...........#.#...#.....#.#...#...#.....#.#.#...#.....#.....#...#.....#.#.#.#...#.#...#.#...#', '#.#.#.#############.###.#######.#.#.###.#.###.#.###########.#.#.###.#####.###.#.###.###.#.#.#.#######.#.#####.###.#####.#.#.#.###.###.#.###.#']
141 141
(65, 113) (77, 97)


In [166]:
Q = deque([(s, 0)])
COST_GRID = [[None for _ in range(len(G[0]))] for _ in range(len(G))]
SEEN = set()

while Q:
    pos, cost = Q.popleft()
    r, c = pos

    if (r, c) in SEEN:
        continue
    else:
        SEEN.add((r, c))
        COST_GRID[r][c] = cost

    for d in dirs:
        rr = r + d[0]
        cc = c + d[1]
        if 0 <= rr < len(G) and 0 <= cc < len(G[rr]):
            if G[rr][cc] == ".":
                Q.append(((rr, cc), cost + 1))

shortcut_savings = defaultdict(int)

DP = {}


def reachable_in_n_steps(r, c, n, d_in=None):

    if (r, c, n, d_in) in DP:
        return DP[r, c, n, d_in]

    reachable = set()
    if n == 0:
        DP[r, c, n, d_in] = {(r, c)}
        return {(r, c)}
    else:
        for d_i, d in enumerate(dirs):
            rr = r + d[0]
            cc = c + d[1]
            if 0 <= rr < len(G) and 0 <= cc < len(G[0]):
                if d_in is not None:
                    forwards = d_i == d_in
                    same_plane = d_in % 2 == d_i % 2
                    backwards = same_plane and not forwards
                if d_in is None or (not backwards):
                    n_r = reachable_in_n_steps(rr, cc, n - 1, d_i)
                    reachable.update(n_r)
        DP[r, c, n, d_in] = reachable
        return reachable


NDP = {}


def reachable_in_at_most_n_steps(r, c, n):

    if (r, c, n) in NDP:
        return NDP[r, c, n]
    if n == 0:
        NDP[r, c, n] = {(r, c)}
        return {(r, c)}
    else:
        reachable = {(r, c)}
        for d_i, d in enumerate(dirs):
            rr = r + d[0]
            cc = c + d[1]
            if 0 <= rr < len(G) and 0 <= cc < len(G[0]):
                n_r = reachable_in_at_most_n_steps(rr, cc, n - 1)
                reachable.update(n_r)
        NDP[r, c, n] = reachable
        return reachable


def cheats_of_distance(N, MINIMUM_SAVING, part2=False):
    shortcut_savings = defaultdict(int)

    for r, row in enumerate(G):
        for c, cell in enumerate(row):
            if part2:
                reachables = reachable_in_at_most_n_steps(r, c, N)
            else:
                reachables = reachable_in_n_steps(r, c, N)
            for rr, cc in reachables:
                man_dist = abs(rr - r) + abs(cc - c)
                if 0 <= rr < len(G) and 0 <= cc < len(G[0]):
                    if COST_GRID[rr][cc] is not None and COST_GRID[r][c] is not None:
                        saving = COST_GRID[rr][cc] - COST_GRID[r][c] - man_dist
                        if saving > 0:
                            shortcut_savings[saving] += 1

    ans = 0
    for saving, count in sorted(shortcut_savings.items(), key=lambda x: -x[1]):
        if saving >= MINIMUM_SAVING:
            ans += count

    print(ans)


cheats_of_distance(2, 100, part2=False)

# 569718 too low.
# 1247457 too high.
cheats_of_distance(20, 100, part2=True)  # should be 285 for testing.

1502
1028136
